In [12]:
import pandas as pd
import datetime
import vk_api
import os
import requests
import json
import random
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import sys

# Подключение к Группе и Беседе и отправка тестового сообщения

In [ ]:
token = '2cd9ff8440bd268b5dacd2902d2752ef8b2b6d6e508401992553a9b885f703cee10ff8ac89a05fe1ae872'

In [14]:
vk_session = vk_api.VkApi(token = token)
vk = vk_session.get_api()

In [26]:
vk.messages.send(
    chat_id = 4,
    random_id = 3,
    message='Кароче, я это через код мучу, тип Бот получается')# Название беседы: ТЕСТ(ключ - 4)

0

In [53]:
# vk.groups.get(user_id='malchik_neprostoy')

# группировка файла (пример задания)

In [31]:
df = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-v-kachanovskij-24/shared/ads_data.csv.zip', compression='zip')

In [34]:
ad_data = df.groupby(['ad_id', 'event'],as_index = False).agg({'user_id':'count'})

In [38]:
ad_data = ad_data.pivot(index = 'ad_id', columns = 'event', values = 'user_id').reset_index()

In [39]:
ad_data = ad_data.fillna(0).assign(ctr = ad_data.click / ad_data.view)

In [42]:
top_ctr = ad_data.query('click > 20 & view > 100').sort_values('ctr').tail(10)

In [44]:
top_ctr.to_csv('top_ctr_data.csv', index = False)

# Загрузка файла на сервер Вк и прикрепление его к сообщению

In [46]:
path = '/mnt/HC_Volume_18315164/home-jupyter/jupyter-v-kachanovskij-24/Lesson7/top_ctr_data.csv'
file_name = 'top_ctr_data.csv'

path_to_file = path
upload_url = vk.docs.getMessagesUploadServer(peer_id = 2000000004)["upload_url"]#Берем информацию о Сервере
file = {'file':(file_name, open(path_to_file, 'rb'))}


response = requests.post(upload_url, files = file)#Загрузка

json_data = json.loads(response.text)#Загрузка
json_data

{'file': '303469778|0|-1|536236|25e3fa1167|csv|403|top_ctr_data.csv|c2f9e346fc49058f43500d24dbf7ae44|45a67c45a5db92fc336cf1533733644b||||eyJkaXNrIjoyMiwicGVlcl9zZW5kZXIiOiItMjEzODE1Njg2In0='}

In [47]:
saved_file = vk.docs.save(file = json_data['file'], title = file_name)#Сохраняем в документы 

In [50]:
attachment = 'doc{}_{}'.format(saved_file['doc']['owner_id'], saved_file['doc']['id'])#Переводим в нужный формат

In [51]:
vk.messages.send(
    chat_id = 4,
    random_id = 4,
    message='Топ объявлений по CTR',
    attachment = attachment)# Название беседы: ТЕСТ(ключ - 4)

0